<a href="https://colab.research.google.com/github/fine-man/cvit-workshop/blob/main/day6/session1/d6-s1-wandb-demo-hands-on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using wandb to track experiments.

Demo task: multi-class image classification using CIFAR10 dataset.

In [ ]:
from sklearn.metrics import average_precision_score
from torch.utils.data import DataLoader
from torchvision import datasets, models
from torchvision import transforms as T
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# The next cell includes-
- Collecting the CIFAR10 dataset and defining data loaders.
- Methods to load model, criterion, optimizer and schedulers.
- Definition of AverageMeter

In [ ]:
# Downloading CIFAR10 dataset
inp_transforms = T.Compose([T.ToTensor(),
                            T.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])])
tgt_transforms = T.Lambda(lambda y: torch.zeros(10, dtype=torch.long).scatter_(0, torch.tensor(y), value=1))
cifar10 = datasets.CIFAR10(root = "/.",
                           transform = inp_transforms,
                           target_transform = tgt_transforms,
                           download = True)

# Defining dataset split (80-20)
train_dataset, val_dataset = torch.utils.data.random_split(cifar10,
                                                           [int(len(cifar10)*0.80), int(len(cifar10)*0.20)])

# Defining the dataloaders
train_dataloader = DataLoader(train_dataset,
                              batch_size=200,
                              shuffle=True)
val_dataloader = DataLoader(val_dataset,
                            batch_size=200,
                            shuffle=False)


# Method to get model based on config param model_type
def get_model(model_type):
    model = None
    if model_type == "pretrained": # Loading pretrained ResNet18 and with updated to final fc layer. 
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(512, 10)
        model = model.to(device)
    elif model_type == "scratch": # Loading a blank ResNet18 which generated 10 outputs.
        model = models.resnet18(num_classes=10)
        model = model.to(device)
    else:
        raise NotImplemented
    return model


# Method to get criterion, optimizer and scheduler based on config params.
def get_criterion_optimizer_scheduler(config, model):
    optim_dct = {
        "adam": optim.Adam,
        "SGD": optim.SGD,
        "RMSprop": optim.RMSprop
    }
    optimizer = optim_dct[config["optimizer"]](model.parameters(), lr=config["lr"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                           factor=0.1,
                                                           patience=config["scheduler_patience"],
                                                           threshold=config["scheduler_thresh"])
    criterion = nn.CrossEntropyLoss()
    return criterion, optimizer, scheduler



# Remainder of this cell includes definition of AverageMeter (can be ignored)
"""
Code taken from Pytorch ImageNet examples
https://github.com/pytorch/examples/blob/main/imagenet/main.py#L375
"""
class Summary():
    NONE = 0
    AVERAGE = 1
    SUM = 2
    COUNT = 3

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f', summary_type=Summary.AVERAGE):
        self.name = name
        self.fmt = fmt
        self.summary_type = summary_type
        self.val_history = list()
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.val_history = list()

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        self.val_history.append(val)

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
    
    def summary(self):
        fmtstr = ''
        if self.summary_type is Summary.NONE:
            fmtstr = ''
        elif self.summary_type is Summary.AVERAGE:
            fmtstr = '{name} {avg:.3f}'
        elif self.summary_type is Summary.SUM:
            fmtstr = '{name} {sum:.3f}'
        elif self.summary_type is Summary.COUNT:
            fmtstr = '{name} {count:.3f}'
        else:
            raise ValueError('invalid summary type %r' % self.summary_type)        
        return fmtstr.format(**self.__dict__)


100%|██████████| 170498071/170498071 [00:05<00:00, 29091127.41it/s]


Extracting /./cifar-10-python.tar.gz to /.


# Following cell includes-
- Defining the train and eval loops.
- Method to trigger training loops based on config parameters.

In [ ]:
# The train function without wandb logging

def train(model, criterion, optimizer, scheduler, epochs, train_dataloader, val_dataloader, device):
    for epoch in range(epochs):
        model.train()
        loss_meter = AverageMeter("train_loss", ":.5f")
        epoch_outs, epoch_tgt = list(), list()
        for data, tgt_vec in tqdm(train_dataloader):
            data, tgt_vec = data.to(device), tgt_vec.to(device)
            targets = torch.argmax(tgt_vec, axis=1)
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out, targets)
            loss_meter.update(loss.item(), data.shape[0])
            loss.backward()
            optimizer.step()
            epoch_outs.append(out)
            epoch_tgt.append(tgt_vec)
        predictions = torch.vstack([torch.softmax(out, axis=1) for out in epoch_outs]).detach().cpu().numpy()
        targets = torch.cat([tgt for tgt in epoch_tgt], dim=0).detach().cpu().numpy()
        ap_score = average_precision_score(targets, predictions)
        eval_loss_meter, eval_ap_score = evaluate(model, criterion, val_dataloader, device)
        data_to_log = {
            "epoch": epoch+1,
            "train_loss": loss_meter.avg,
            "eval_loss": eval_loss_meter.avg,
            "train_ap_score": ap_score,
            "eval_ap_score": eval_ap_score,
            "lr": optimizer.state_dict()["param_groups"][0]["lr"],
        }
        scheduler.step(eval_loss_meter.avg)
        wandb.log(data_to_log)
        print(data_to_log)


@torch.no_grad()
def evaluate(model, criterion, val_dataloader, device):
    model.eval()
    loss_meter = AverageMeter("eval_loss", ":.5f")
    epoch_outs, epoch_tgt = list(), list()
    for data, tgt_vec in val_dataloader:
        data, tgt_vec = data.to(device), tgt_vec.to(device)
        targets = torch.argmax(tgt_vec, axis=1)
        out = model(data)
        loss = criterion(out, targets)
        loss_meter.update(loss.item(), data.shape[0])
        epoch_outs.append(out)
        epoch_tgt.append(tgt_vec)
    predictions = torch.vstack([torch.softmax(out, axis=1) for out in epoch_outs]).detach().cpu().numpy()
    targets = torch.cat([tgt for tgt in epoch_tgt], dim=0).detach().cpu().numpy()
    ap_score = average_precision_score(targets, predictions)
    return loss_meter, ap_score


def trigger_training(config):
    model = get_model(config["model_type"])
    criterion, optimizer, scheduler = get_criterion_optimizer_scheduler(config, model)
    epochs = config["num_epochs"]

    train(model, criterion, optimizer, scheduler, epochs, train_dataloader, val_dataloader, device)


# Complete the config file, edit the cells in this notebook to log data to wandb and trigger training loops!

In [ ]:
# Fill the Config file below and log the experiment at wandb
config = {
    "lr": 0.001, 
    "model_type": "scratch", # pretrained/scratch
    "optimizer": "adam", # adam/SGD/RMSprop
    "criterion": "ce",
    "scheduler_patience": 3,
    "scheduler_thresh": 0.001,
    "num_epochs": 20, # CHANGE
    "gpu_id": 0,
    "wandb_run_name": "E1.le-3.scratch.adam.ce.sp3.ste-3.epoch20" ### FILL YOUR NAME HERE
}


In [ ]:
trigger_training(config)

100%|██████████| 200/200 [00:18<00:00, 11.03it/s]


{'epoch': 1, 'train_loss': 1.423871293067932, 'eval_loss': 1.2828774237632752, 'train_ap_score': 0.5148937157897504, 'eval_ap_score': 0.6182428086121007, 'lr': 0.001}


100%|██████████| 200/200 [00:18<00:00, 11.04it/s]


{'epoch': 2, 'train_loss': 1.0189158609509468, 'eval_loss': 1.0337530875205994, 'train_ap_score': 0.6998442689744173, 'eval_ap_score': 0.7181850587863515, 'lr': 0.001}


100%|██████████| 200/200 [00:18<00:00, 11.10it/s]


...

{'epoch': 18, 'train_loss': 0.006280314652249217, 'eval_loss': 1.0920600616931915, 'train_ap_score': 0.9999834928110873, 'eval_ap_score': 0.8342009985411281, 'lr': 1e-05}


100%|██████████| 200/200 [00:18<00:00, 11.00it/s]


{'epoch': 19, 'train_loss': 0.005880813945550472, 'eval_loss': 1.0939810061454773, 'train_ap_score': 0.9999918202908841, 'eval_ap_score': 0.8341298065402467, 'lr': 1.0000000000000002e-06}


100%|██████████| 200/200 [00:18<00:00, 10.87it/s]


{'epoch': 20, 'train_loss': 0.005865726522170007, 'eval_loss': 1.0924844241142273, 'train_ap_score': 0.9999917212461277, 'eval_ap_score': 0.8343630855725441, 'lr': 1.0000000000000002e-06}


# WandB Steps

In [ ]:
# installing wandb
!pip install wandb

In [ ]:
### Step 1: Import WandB in your code

import wandb

### Step 1 ends

In [ ]:
### Step 2:
# Initiate wandb in your script. The moment we trigger wandb.init(), an active
# socket connection is established between your machine and wandb server.
# We specify the entity (wandb username) and project (which wandb project to use for logging)

wandb.init(entity = "vanshg",   # wandb username. (NOT REQUIRED ARG. ANYMORE, it fetches from initial login)
           project = "wandb_demo", # wandb project name. New project will be created if given project is missing.
           config = config         # Config dict
          )
wandb.run.name = config["wandb_run_name"]

### Step 2 ends.

In [ ]:
### Step 3: Trigger wandb log
# This step is responsible for sending the logs to wandb

wandb.log(data_to_log)

### Step 3 ends.

NameError: name 'data_to_log' is not defined

In [ ]:
### Step 4 (Optional)
# This closes the active socket connection to wandb server. Optional since wandb destructor does the same.

wandb.finish()

### Step 4 ends.


# WandB sweeps related steps

In [ ]:
### Step 1:
# Create a WandB sweep config file.
# This config file will be used at the WandB website to initialize a sweep server
program: "demo.py"
method: "grid"
metric:
  name: "eval_ap_score"
  goal: "maximize"
parameters:
    criterion:
      value: "ce"
    gpu_id:
      value: 0
    lr:
      values: [0.1, 0.001, 0.0001]
    model_type:
      values: ["scratch", "pretrained"]
    num_epochs:
      value: 25
    optimizer:
      values: ["adam", "SGD", "RMSprop"]
    scheduler_patience:
      value: 3
    scheduler_thresh:
      value: 0.01

        
### A sample sweep config file if bayes method is used-
# program: wandb_demo.py
# method: bayes
# metric:
#   name: "eval_ap_score"
#   goal: maximize
# parameters:
#   lr:
#     distribution: uniform
#     min: 0.00001
#     max: 0.1
#   criterion:
#     distribution: categorical
#     value:
#       - ce
#   optimizer:
#     distribution: categorical
#     values:
#       - adam
#       - SGD
#       - RMSprop
#   model_type:
#     distribution: categorical
#     values:
#       - pretrained
#       - scratch
#   num_epochs:
#     value:
#       - 30
#   scheduler_thresh:
#     distribution: uniform
#     min: 0.001
#     max: 0.01
#   scheduler_patience:
#     distribution: int_uniform
#     min: 2
#     max: 10


SyntaxError: invalid syntax (3096734612.py, line 6)

In [ ]:
sweep_configuration_grid = {
    'program': 'demo.py',
    'method': "grid",
    'metric': {
        'name': "eval_ap_score",
        'goal': "maximize"
    },
    'parameters': {
        'criterion': {'value': "ce"},
        'gpu_id': {'value': 0},
        'lr': {'values': [0.1, 0.001, 0.0001]},
        'model_type': {'values': ["scratch", "pretrained"]},
        'num_epochs': {'value': 25},
        'optimizer': {'values': ["adam", "SGD", "RMSprop"]},
        'scheduler_patience': {'value': 3},
        'scheduler_thresh': {'value': 0.01}
    }
}

sweep_grid_id = wandb.sweep(
    sweep=sweep_configuration_grid,
    project='wandb_demo'
)

print(f'Sweep grid id: {sweep_grid_id}')

Create sweep with ID: xmxmiut3
Sweep URL: https://wandb.ai/vanshg/wandb_demo/sweeps/xmxmiut3
Sweep grid id: xmxmiut3


In [ ]:
### Step 2
# After using the above config on wandb website, you will get a sweep id in return.
# E.g. sweep id- dhruv_sri/wandb_demo/hbyp0tl8
#
# Add the following agent line in your code-
# Use the generated sweep id in the below code

wandb.agent(sweep_id="xmxmiut3", function=sweep_agent_manager, count=100)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: lqbja0b9 with config:
wandb: 	criterion: ce
wandb: 	gpu_id: 0
wandb: 	lr: 0.1
wandb: 	model_type: scratch
wandb: 	num_epochs: 25
wandb: 	optimizer: adam
wandb: 	scheduler_patience: 3
wandb: 	scheduler_thresh: 0.01


<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


100%|██████████| 200/200 [00:19<00:00, 10.24it/s]


{'epoch': 1, 'train_loss': 2.832748358249664, 'eval_loss': 2.304744629859924, 'train_ap_score': 0.10221365123961627, 'eval_ap_score': 0.10350224513945994, 'lr': 0.1}


100%|██████████| 200/200 [00:18<00:00, 11.06it/s]


...

100%|██████████| 200/200 [00:17<00:00, 11.15it/s]


{'epoch': 10, 'train_loss': 0.018624372686317656, 'eval_loss': 1.0135586738586426, 'train_ap_score': 0.9996556250135491, 'eval_ap_score': 0.8936858946544941, 'lr': 0.0001}


100%|██████████| 200/200 [00:18<00:00, 11.09it/s]


{'epoch': 11, 'train_loss': 0.009322105975297746, 'eval_loss': 1.1563803899288176, 'train_ap_score': 0.9999304934426526, 'eval_ap_score': 0.8928196811225566, 'lr': 0.0001}


 76%|███████▌  | 152/200 [00:13<00:04, 10.58it/s]

In [ ]:
### Step 3
# Notice in above command we mentioned an argument named "function"
# Wandb agents must trigger a function where they can initiate a socket to wandb and get a config.
# So, we will use the following sweep_agent_manager function here-

def sweep_agent_manager():
    wandb.init()
    config = dict(wandb.config)
    run_name = f"{config['model_type']}_{config['optimizer']}_{config['lr']}"
    wandb.run.name = run_name
    trigger_training(config)


In [ ]:
### Done.
# Now execute your training script on multiple machines.
# Each run will request the config file from wandb and related experiments will be logged.
# 
# NOTE!! wandb.log(data_to_log) must be present inside the code!! Else there is no meaning to sweep.


# ------------------------------ Ends ------------------------------